In [ ]:
import os
import pandas as pd
from faker import Faker 
import datetime
import csv

In [ ]:
fake = Faker()

In [ ]:
def fake_person(id=0):
    first_name = fake.first_name()
    last_name = fake.last_name()
    return {
                "id:ID": "person_id_{id}".format(id=id),
                "first_name": first_name,
                "last_name": last_name,
                "dob:date" : fake.date(pattern="%Y-%m-%d", end_datetime=datetime.date(1995, 1,1)),
                "email": str.lower(f"{first_name}.{last_name}@fake_domain-2.com"),
                "city" : fake.city(),
                "country" : fake.country(),
            }

In [ ]:
def fake_person_generation(records, id_start=0):
    result = []
    id = 0
    for i in range(records):
        result.append(fake_person(id + id_start))
        id += 1
    return result

In [ ]:
df_persons = pd.DataFrame(fake_person_generation(10))
df_persons.head()

In [ ]:
df_persons.columns.to_list()

In [ ]:

def fake_person_csv(records, id_start=0, f=0):
    with open("persons{f}.csv".format(f=f), 'wt') as csvFile:
        writer = csv.DictWriter(csvFile, fieldnames=df_persons.columns.to_list())
        writer.writeheader()
        id = 0
        for i in range(records):
            writer.writerow(fake_person(id+id_start))
            id += 1


In [ ]:
rows_per_file=1_000_000;
for f in range(100):
    fake_person_csv(rows_per_file, id_start=rows_per_file*f, f=f)
    f += 1
